In [1]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import json
import time
from credentials import *

In [2]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id,
                                                           client_secret=client_secret))

In [3]:
def search_song(title, artist):
    
    query = "tracks: "+title+" artist: "+artist
    #print(query)

    try:
        result = sp.search(query, limit=1)
        song_id = result['tracks']['items'][0]['id']
    except:
        song_id = "not_found"

    return song_id  

In [4]:
search_song("Hello", "Adele")

'1AynHbA5tcynFDplpCY5lz'

In [7]:
df1 = pd.read_csv('/Users/jxhn/Desktop/IRONHACK/Week_6/Day_1/Morning/lab-web-scraping-single-page/hot100.csv')
df2 = pd.read_csv('/Users/jxhn/Desktop/IRONHACK/Week_6/Song_recommender/Song-Recommender/nothot100.csv')
df2.isnull().sum()

Artist    0
Title     2
dtype: int64

In [8]:
df2 = df2.drop(65)

In [9]:
# create groups for 50 song for df1 and df2
chunks_df1 = [df1.iloc[i:i+50] for i in range(0, len(df1), 50)]

In [10]:
# create groups fo 50 song for df1
chunks_df2 = [df2[i:i + 50] for i in range(0, len(df2), 50)]

In [11]:
#create the list of id's of df1
song_ids_df1 = []
for chunk in chunks_df1:
    for index, row in chunk.iterrows():
        song_id = search_song(row["Title"], row["Artist"])
        song_ids_df1.append(song_id)
    time.sleep(20)
print('finished')

finished


In [12]:
#create the list of id's of df2
song_ids_df2 = []
for chunk in chunks_df2:
    for index, row in chunk.iterrows():
        song_id = search_song(row["Title"], row["Artist"])
        song_ids_df2.append(song_id)
    time.sleep(20)
print('finished')

TypeError: can only concatenate str (not "float") to str

In [ ]:
len(song_ids_df1)

In [ ]:
 def get_audio_features(list_of_song_ids):
    
    chunks = [list_of_song_ids[i:i + 50] for i in range(0, len(list_of_song_ids), 50)]
    
    # Create a dictionary to store the audio features
    audio_features_dict = {
        'id': [],
        'energy': [],
        'key': [],
        'acousticness': [],
        'instrumentalness': [],
        'liveness': [],
        'tempo': [],
        'type': [],
        'uri': [],
        'track_href': [],
        'analysis_url': [],
        'duration_ms': [],
    }
    
    # Retrieve the audio features for each chunk
    for chunk in chunks:
        audio_features = sp.audio_features(chunk)
        
        # Append the audio features to the dictionary
        for feature in audio_features:
            if feature:
                for key in audio_features_dict.keys():
                    audio_features_dict[key].append(feature[key])
        
        # Wait 20 seconds before next API call
        time.sleep(20)

    # Create a DataFrame from the dictionary
    df = pd.DataFrame(audio_features_dict)

    return df

In [ ]:
df3 = get_audio_features(song_ids_df1)
len(df3)

In [ ]:
results_hot100 = pd.concat([df1, df3], axis=1)
results_hot100.head()

In [ ]:
results_hot100.to_csv('results_hot100.csv', index=False)

In [ ]:
len(song_ids_df2)

In [ ]:
#to get the index which has 'not_found'
index_list = []
for i in range(0,2499):
    if song_ids_df2[i] == 'not_found':
        index_list.append(i)
index_list 

In [ ]:
list_of_valid_id = []
for x in song_ids_df2:
    if x != 'not_found':
        list_of_valid_id.append(x)
song_ids_df2 = list_of_valid_id

In [ ]:
len(song_ids_df2)

In [ ]:
df4 = get_audio_features(song_ids_df2)

In [ ]:
df4.shape

In [ ]:
results_nothot100 = pd.concat([df2, df4], axis=1)
results_nothot100

In [ ]:
results_nothot100.dropna(subset=['id'], inplace=True)
results_nothot100

In [ ]:
results_nothot100.to_csv('results_nothot100.csv', index=False)